In [1]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [2]:
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

In [3]:
import pandas as pd; 
df = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data", delim_whitespace=True, names=["mpg","cylinders","displacement","horsepower","weight","acceleration","model_year","origin","car_name"], na_values='?')

 

/tmp/ipykernel_4994/255274924.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data", delim_whitespace=True, names=["mpg","cylinders","displacement","horsepower","weight","acceleration","model_year","origin","car_name"], na_values='?')


In [4]:
df


,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790.0,15.6,82,1,ford mustang gl
394,44.0,4,97.0,52.0,2130.0,24.6,82,2,vw pickup
395,32.0,4,135.0,84.0,2295.0,11.6,82,1,dodge rampage
396,28.0,4,120.0,79.0,2625.0,18.6,82,1,ford ranger


In [5]:
df.isnull().sum()

mpg             0
cylinders       0
displacement    0
horsepower      6
weight          0
acceleration    0
model_year      0
origin          0
car_name        0
dtype: int64

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load dataset directly from UCI repository
df = pd.read_csv(
    "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data",
    delim_whitespace=True,
    names=["mpg","cylinders","displacement","horsepower","weight","acceleration","model_year","origin","car_name"],
    na_values='?'
)

# Fill missing values with column mean
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].mean())

# Separate features and target
X = df[['cylinders','displacement','horsepower','weight','acceleration','model_year','origin']]
y = df['mpg']



# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape)


/tmp/ipykernel_4994/1989235481.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(


(318, 7) (80, 7)


In [7]:
# Linear Regression
import mlflow
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from mlflow.models import infer_signature
import numpy as np
 
mlflow.set_tracking_uri("http://4.246.173.149:5000")
mlflow.set_experiment("MpgEstimationExperiment")
 
with mlflow.start_run(run_name="LinearRegression"):
    model = LinearRegression()
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
 
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    mae = mean_absolute_error(y_test, predictions)
 
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mae", mae)
    mlflow.log_param("model_type", "Linear Regression")
 
    signature = infer_signature(X_train, model.predict(X_train))
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="model",
        signature=signature,
        input_example=X_train,
        registered_model_name="MpgEstimator"
    )


/home/azureuser/anaconda3/lib/python3.9/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/08/11 11:01:38 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/11 11:01:40 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model U

🏃 View run LinearRegression at: http://4.246.173.149:5000/#/experiments/2/runs/e653111e19744eedb8a8024448d68f3b
🧪 View experiment at: http://4.246.173.149:5000/#/experiments/2


In [8]:
# Gradient Boosting Regressor
import mlflow
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from mlflow.models import infer_signature
import numpy as np

mlflow.set_tracking_uri("http://4.246.173.149:5000")
mlflow.set_experiment("MpgEstimationExperiment")

with mlflow.start_run(run_name="GradientBoostingRegressor"):
    model = GradientBoostingRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=3,
        random_state=42
    )
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    mae = mean_absolute_error(y_test, predictions)

    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mae", mae)
    mlflow.log_param("model_type", "Gradient Boosting Regressor")

    signature = infer_signature(X_train, model.predict(X_train))
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="model",
        signature=signature,
        input_example=X_train,
        registered_model_name="MpgEstimator"
    )


/home/azureuser/anaconda3/lib/python3.9/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/08/11 11:01:41 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/11 11:01:43 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model U

🏃 View run GradientBoostingRegressor at: http://4.246.173.149:5000/#/experiments/2/runs/7ff457d2871f414ca1fe527c71c0ae0b
🧪 View experiment at: http://4.246.173.149:5000/#/experiments/2


In [12]:
# Random Forest Regressor
import mlflow
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from mlflow.models import infer_signature
import numpy as np

mlflow.set_tracking_uri("http://4.246.173.149:5000")
mlflow.set_experiment("MpgEstimationExperiment")

with mlflow.start_run(run_name="RandomForestRegressor"):
    model = RandomForestRegressor(
        n_estimators=100,
        max_depth=None,
        random_state=42
    )
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    mae = mean_absolute_error(y_test, predictions)

    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mae", mae)
    mlflow.log_param("model_type", "Random Forest Regressor")

    signature = infer_signature(X_train, model.predict(X_train))
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="model",
        signature=signature,
        input_example=X_train,
        registered_model_name="MpgEstimator"
    )


/home/azureuser/anaconda3/lib/python3.9/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/08/11 11:03:01 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/11 11:03:03 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model U

🏃 View run RandomForestRegressor at: http://4.246.173.149:5000/#/experiments/2/runs/d68589d457304c76a1f40bf91a6bcdfe
🧪 View experiment at: http://4.246.173.149:5000/#/experiments/2


In [13]:
# K-Nearest Neighbors Regressor
import mlflow
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from mlflow.models import infer_signature
import numpy as np

mlflow.set_tracking_uri("http://4.246.173.149:5000")
mlflow.set_experiment("MpgEstimationExperiment")

with mlflow.start_run(run_name="KNNRegressor"):
    model = KNeighborsRegressor(
        n_neighbors=5,
        weights='uniform'
    )
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    mae = mean_absolute_error(y_test, predictions)

    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mae", mae)
    mlflow.log_param("model_type", "KNN Regressor")

    signature = infer_signature(X_train, model.predict(X_train))
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="model",
        signature=signature,
        input_example=X_train,
        registered_model_name="MpgEstimator"
    )


/home/azureuser/anaconda3/lib/python3.9/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/08/11 11:03:56 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/11 11:03:58 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model U

🏃 View run KNNRegressor at: http://4.246.173.149:5000/#/experiments/2/runs/e092e814c83f41f8aa80cb0964c2491a
🧪 View experiment at: http://4.246.173.149:5000/#/experiments/2


In [15]:
# XGBoost Regressor
import mlflow
!pip install xgboost

from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from mlflow.models import infer_signature
import numpy as np

mlflow.set_tracking_uri("http://4.246.173.149:5000")
mlflow.set_experiment("MpgEstimationExperiment")

with mlflow.start_run(run_name="XGBoostRegressor"):
    model = XGBRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=3,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    mae = mean_absolute_error(y_test, predictions)

    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mae", mae)
    mlflow.log_param("model_type", "XGBoost Regressor")

    signature = infer_signature(X_train, model.predict(X_train))
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="model",
        signature=signature,
        input_example=X_train,
        registered_model_name="MpgEstimator"
    )


     |████████████████████████████████| 223.6 MB 4.3 kB/s  eta 0:00:01
     |███████████████████████████▊    | 279.9 MB 96.5 MB/s eta 0:00:011

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 322.5 MB 20 kB/s 


/home/azureuser/anaconda3/lib/python3.9/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/08/11 11:06:28 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/11 11:06:30 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model U

🏃 View run XGBoostRegressor at: http://4.246.173.149:5000/#/experiments/2/runs/653a2ef1bc704c93aee95f0519ef06a3
🧪 View experiment at: http://4.246.173.149:5000/#/experiments/2


In [16]:
import mlflow

# Get experiment
experiment_name = "MpgEstimationExperiment"
experiment = mlflow.get_experiment_by_name(experiment_name)

# Fetch all runs
runs_df = mlflow.search_runs(experiment_ids=[experiment.experiment_id])

# Show only model name and RMSE
result_df = runs_df[["tags.mlflow.runName", "metrics.rmse", "metrics.mae"]]
result_df = result_df.sort_values(by="metrics.rmse", ascending=True)
print(result_df)


         tags.mlflow.runName  metrics.rmse  metrics.mae
2      RandomForestRegressor      2.143801     1.592725
3  GradientBoostingRegressor      2.250585     1.702208
0           XGBoostRegressor      2.351331     1.757389
4           LinearRegression      2.862770     2.253388
1               KNNRegressor      3.547022     2.769250


In [19]:
import mlflow
import pickle

# Connect to your MLflow tracking server
mlflow.set_tracking_uri("http://4.246.173.149:5000")

# Registered model details
reg_model_name = "MpgEstimator"
model_version = 10
model_uri = f"models:/{reg_model_name}/{model_version}"

# Load the model from MLflow
loaded_model = mlflow.sklearn.load_model(model_uri)
print(f"Loaded model: {reg_model_name} v{model_version}")

# Save to local file as lr_model.bin
with open("lr_model.bin", "wb") as f_out:
    pickle.dump(loaded_model, f_out)

print("Model saved locally as lr_model.bin")

Loaded model: MpgEstimator v10
Model saved locally as lr_model.bin
